Source for the model : https://github.com/Raocp/PINN-elastodynamics/blob/master/ElasticWaveInfinite/ElasticWave.py

In [1]:
import numpy as np
import time
from pyDOE import lhs
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import shutil
import pickle
import math
import scipy.io

In [3]:
# Need pretraining!! (i.e. train for 10s -> 15s -> 25s)
PI = math.pi
MAX_T = 20.0

In [4]:
# Domain bounds
lb = np.array([0.0, 0.0, 0.0])
ub = np.array([30, 30, MAX_T])

In [5]:
# Network configuration
uv_layers = [3] + 8*[80] + [7]

# Properties of source
xc_src = 15.0
yc_src = 15.0
r_src = 2.0

# Num of lower/upper BC points
N_lwb = 101
N_upb = 101

# Num of left/right BC points
N_lfb = 101
N_rtb = 101

# Num of collocation point in x, y, t
N_f = 120000

N_t = int(MAX_T*4+1)  # 4 frames per second

In [ ]:
    # Initial condition point for u, v
    x_IC, y_IC, t_IC = CartGrid(xmin=0, xmax=30, ymin=0, ymax=30, tmin=0, tmax=0, num=101, num_t=1)
    IC = np.concatenate((x_IC, y_IC, t_IC), 1)
    # IC = DelSrcPT(IC, xc=xc_src, yc=yc_src, r=r_src)

    # Upper boundary point
    x_up = np.linspace(0, 30, 150)
    t_up = np.linspace(0, MAX_T, 201)
    x_up, t_up = np.meshgrid(x_up, t_up)
    x_up = x_up.flatten()[:, None]
    t_up = t_up.flatten()[:, None]
    y_up = np.zeros((x_up.size, 1))
    y_up.fill(30.0)
    UP = np.concatenate((x_up, y_up, t_up), 1)

    # Collocation point
    XYT_c = lb + (ub - lb) * lhs(3, N_f)
    XYT_c_ext = np.array([xc_src - r_src -1, yc_src - r_src -1, 0.0]) + np.array([2*(r_src+1), 2*(r_src+1), MAX_T]) * lhs(3, 10000)   # Refinement around source
    # XYT_c_ext = np.array([0, yc_src - r_src - 1, 0.0]) + np.array([30, r_src + 1, MAX_T]) * lhs(3, 20000)
    XYT_c = np.concatenate((XYT_c, XYT_c_ext),axis=0)
    XYT_c = DelSrcPT(XYT_c, xc=xc_src, yc=yc_src, r=r_src)

    # plt.scatter(XYT_c[:, 0], XYT_c[:, 1], marker='o', alpha=0.2 )
    # plt.show()

    # Wave source point in the middle, x=15.0, y=30.0
    tsh = 3.0
    ts = 3.0
    Amp = 1.0
    xx, yy, theta = GenCirclePT(xc=xc_src, yc=yc_src, r=r_src, N_PT=200)   # N_PT=500
    tt = np.linspace(0, MAX_T, 353)
    tt = tt[1:]
    x_SRC, t_SRC = np.meshgrid(xx, tt)
    y_SRC, _     = np.meshgrid(yy, tt)
    x_SRC = x_SRC.flatten()[:, None]
    y_SRC = y_SRC.flatten()[:, None]
    t_SRC = t_SRC.flatten()[:, None]
    amplitude = Amp*(2*PI**2*(t_SRC-ts)**2/tsh**2-1)*np.exp(-PI**2*(t_SRC-ts)**2/tsh**2)
    u_SRC = amplitude*(x_SRC-xc_src)/r_src
    v_SRC = amplitude*(y_SRC-yc_src)/r_src
    SRC = np.concatenate((x_SRC, y_SRC, t_SRC, u_SRC, v_SRC), 1)


    # plt.scatter(t_SRC,amplitude, marker='o', alpha=0.2 )
    # plt.show()

    # plt.scatter(x_SRC,y_SRC, marker='o', alpha=0.2 )
    # plt.show()

    # plt.scatter(XYT_c[:,0:1], XYT_c[:,1:2], marker='o', alpha=0.2 )
    # plt.scatter(SRC[:, 0:1], SRC[:, 1:2], marker='o', alpha=0.2, color='red')
    # plt.scatter(UP[:, 0:1], UP[:, 1:2], marker='o', alpha=0.2)
    # plt.show()

    # Visualize ALL the training points
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(XYT_c[:,0:1], XYT_c[:,1:2], XYT_c[:,2:3], marker='o', alpha=0.1, s=2, color='blue')
    # ax.scatter(SRC[:, 0:1], SRC[:, 1:2], SRC[:, 2:3], marker='o', alpha=0.3, s=2,color='red')
    # ax.scatter(UP[:, 0:1], UP[:, 1:2], UP[:, 2:3], marker='o', alpha=0.2)
    # ax.scatter(IC[:, 0:1], IC[:, 1:2], IC[:, 2:3], marker='o', alpha=0.2)
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('T axis')
    plt.show()